In [ ]:
import geopandas as gpd
import pandas as pd
import requests as req

# Path to the .tab file inside the zip
tab_file = "rakennukset_Helsinki_06_2012_wgs84.TAB"

# Read the .tab file using geopandas
df = gpd.read_file(tab_file)

In [ ]:
df = df[["Osoite", "C_VALMPVM", "C_KAYTTARK", "I_PYRAKNRO", "C_KUNTA", "I_KOKALA", "I_KERRLKM", "C_LAMMTAPA", "C_POLTAINE", "I_RAKTILAV", "C_RAKEAINE"]]

df.sort_values(by="C_VALMPVM", ascending=False, inplace=True)

df.drop_duplicates(subset=["Osoite"], inplace=True)

df = df[2:]

df = df[
    (df["Osoite"] != "") &
    (df["C_VALMPVM"] > "1800-01-01") &
    (df["C_LAMMTAPA"] != "") &
    (df["C_RAKEAINE"] != "") &
    (df["C_POLTAINE"] != "")
]

df
